In [0]:
# Import the function from the src module
%run "/Workspace/Users/lucasmatborges@gmail.com/CaseBEESAmbev/Databricks/src/silver_function"

In [0]:

from pyspark.sql import Row
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# --------------------------
# TEST 1: Fields are trimmed
# --------------------------
def test_trim_columns():
    data = [
        Row(
            address_1=" 100 Main St ",
            address_2=None,
            address_3=None,
            brewery_type=" micro ",
            city=" Austin ",
            country=" USA ",
            id=" 123 ",
            latitude="40.1",
            longitude="-90.2",
            name=" Test Brewery ",
            phone=" 5551234 ",
            postal_code=" 78701 ",
            state=" TX ",
            state_province=" Texas ",
            street=" 100 Main St ",
            website_url=" http://test.com "
        )
    ]

    df_bronze = spark.createDataFrame(data)

    df_silver = transform_silver(df_bronze, "2025-11-23")
    row = df_silver.first()

    assert row.address_1 == "100 Main St"
    assert row.brewery_type == "micro"
    assert row.country == "USA"
    assert row.state == "TX"


# ---------------------------------------------
# TEST 2: Latitude/Longitude are correctly cast
# ---------------------------------------------
def test_lat_long_casting():
    data = [
        Row(
            address_1="A",
            address_2=None,
            address_3=None,
            brewery_type="micro",
            city="Austin",
            country="USA",
            id="1",
            latitude="40.123",
            longitude="-80.654",
            name="Brewery",
            phone="123",
            postal_code="00000",
            state="TX",
            state_province="Texas",
            street="A",
            website_url="http://x.com"
        )
    ]

    df_bronze = spark.createDataFrame(data)
    df_silver = transform_silver(df_bronze, "2025-11-23")
    row = df_silver.first()

    assert isinstance(row.latitude, float)
    assert row.latitude == 40.123
    assert row.longitude == -80.654